# Display map with GeoJSON layers

https://ipyleaflet.readthedocs.io/en/latest/api_reference/geo_json.html


In [1]:
import json
import os
import sys

from ipyleaflet import Map, Marker, GeoJSON

import pandas as pd

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

In [2]:
# Change to parent directory from "jupyter" notebook folder
os.chdir(os.pardir)

In [3]:
# Specify the locality whose OSM file we want to load

locality        = 'Mount Eliza'
locality_detect = 'Mount Eliza Sample'

In [4]:
margin_filename = os.path.join('data_sources', 'Locality_' + locality_detect.replace(' ', '_') + '_margin.geojson')

with open(margin_filename) as margin_file:
    margin_data = json.load(margin_file)
    margin_file.close()

margin_layer = GeoJSON(data=margin_data) 

In [5]:
margin_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[145.092, -38.178],
       [145.092, -38.16],
       [145.112, -38.16],
       [145.112, -38.178],
       [145.092, -38.178]]]]},
   'properties': {'name': 'bounding box'}}]}

In [ ]:
locality_filename = os.path.join('data_sources', 'LGA_boundary_' + locality.replace(' ', '_') + '.geojson')

with open(locality_filename) as locality_file:
    locality_data = json.load(locality_file)
    locality_file.close()

locality_layer = GeoJSON(data=locality_data) 

In [ ]:
pbn_existing_filename = os.path.join('data_sources', 'PBN_Existing.geojson')

with open(pbn_existing_filename) as pbn_file:
    pbn_existing_data = json.load(pbn_file)
    pbn_file.close()
    
pbn_existing_layer = GeoJSON(data=pbn_existing_data, style={'color': 'red'})

In [ ]:
pbn_planned_filename = os.path.join('data_sources', 'PBN_Planned.geojson')

with open(pbn_planned_filename) as pbn_file:
    pbn_planned_data = json.load(pbn_file)
    pbn_file.close()
    
pbn_planned_layer = GeoJSON(data=pbn_planned_data, style={'color': 'orange'})

In [ ]:
osm_filename = os.path.join('data_sources', 'Locality_OSM_cycleway_' + locality.replace(' ', '_') + '.geojson')

with open(osm_filename) as osm_file:
    osm_data = json.load(osm_file)
    osm_file.close()
    
osm_layer = GeoJSON(data=osm_data, style={'color': 'green'})

In [ ]:
detect_filename = os.path.join('detections', locality_detect.replace(' ', '_'), 'detected_points.geojson')

with open(detect_filename) as detect_file:
    detect_data = json.load(detect_file)
    detect_file.close()
    
detect_layer = GeoJSON(data=detect_data, style={'color': 'pink'})

In [6]:
def flatten_coordinates(coordinates):
    coordinates_out = []
    
    if type(coordinates[0]) is list:
        if type(coordinates[0][0]) is list:
            for item in coordinates:
                coordinates_out = coordinates_out + flatten_coordinates(item)
            return coordinates_out
        else:
            return coordinates
    else:
        return coordinates

def get_geojson_centroid(data):
    lat_min =  999.9
    lat_max = -999.9
    lon_min =  999.9
    lon_max = -999.9
    
    for feature in data['features']:
        geometry = feature['geometry']
        coordinates = geometry['coordinates']
        
        flat_coordinates = flatten_coordinates(coordinates)
        
        for point in flat_coordinates:
            
            lat_point = point[1]
            lon_point = point[0]
            
            if lon_point < lon_min:
                lon_min = lon_point
            if lon_point > lon_max:
                lon_max = lon_point
            if lat_point < lat_min:
                lat_min = lat_point
            if lat_point > lat_max:
                lat_max = lat_point
    
    return [((lat_min + lat_max) / 2), ((lon_min + lon_max) / 2)]

In [ ]:
center = get_geojson_centroid(locality_data)
center

In [7]:
center = get_geojson_centroid(margin_data)
center

[-38.169, 145.102]

In [8]:
zoom = 12

In [9]:
m_detect = Map(center=center, zoom=zoom)
m_detect.add_layer(margin_layer)

detection_log = 'detections\Mount_Eliza_Sample_20m\detection_log.csv'

df = pd.read_csv(detection_log)

for index, row in df.iterrows():
    marker = Marker(location=[row['lat'], row['lon']], draggable=False)
    m_detect.add_layer(marker)

m_detect

Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [ ]:
m_pbn = Map(center=center, zoom=zoom)
m_pbn.add_layer(locality_layer)
m_pbn.add_layer(pbn_planned_layer)
m_pbn.add_layer(pbn_existing_layer)
m_pbn

In [ ]:
m_osm = Map(center=center, zoom=zoom)
m_osm.add_layer(locality_layer)
m_osm.add_layer(osm_layer)
m_osm